In [17]:
import numpy as np
from numpy import transpose
from numpy.linalg import inv, det
from scipy.stats import norm

In [22]:
def ridgeFair(X, Y, k, d, _delta, T, _lambda):
    """
    Simulates T rounds of ridgeFair.
    
    :param X: a 3-axis (T, k, d) ndarray of d-dimensional context vectors for each
              time-step and arm
    :param Y: a T x k ndarray of reward function output for each context vector
    :param k: the number of arms
    :param d: the number of features
    :param _delta: confidence parameter
    :param T: the number of iterations
    :param _lambda:  
    """
    picks = []
    for t in itertools.count(): # for t >= 1
        for i in range(k): # for 1 <= i <= k
            R = 1
            intervals = []
            X_i = X[:i] # design matrix (taking all of axis i from X?)
            Y_i = Y[:i] # sem with Y
            Xti = X[:t+1,i] # feature vector for arm i in round t
            
            _X_iT = transpose(X_i) # transpose of X_i
            _idenD = np.identity(d) # identity matrix of dimention d x d
            V_it = _x_iT.dot(X_i)+(_lambda*_idenD) # computing V_it as line 5
            
            B_it = inv(V_it).dot(_X_iT).dot(Y_i) # computing line 6
            y_ti = transpose(Xti).dot(B_it) #computing line 7
            
            _V_itI = inv(V_it) # inverse of V_it
            _Wti1 = norm(Xti)*_V_itI # computing first part of W_ti
            _Wti2 = R*sqrt(d*log((1+(t/_lambda))/_delta))+sqrt(_lambda) # computing second part of W_ti
            W_ti = _Wti1*_Wti2 # computing W_ti as line 8
            
            intervals.append([yh_t_i - w_t_i, yh_t_i + w_t_i]) # line 9
            picks.append(np.argmax(np.array(intervals)[:,1]) if intervals else np.random.randint(0,k)) # line 10
            
            # let S_t be the set of actions chained to i^t?
            # line 15 best = [Y[i].max() for i in range(2, T)]
            # line 16 performance = [Y[t][picks[t-2]] for t in range(2, T)]
            # line 17 update X_i, Y_i
            